In [ ]:
# ================= Kaggle Setup for vsearch + Kraken2 =================
import os

# Update apt repo
!apt-get update -y

# Install dependencies
!apt-get install -y vsearch kraken2 fastp python3-pip build-essential g++ cmake

# Install Bracken (download from GitHub) - Optional, but keeping for completeness
bracken_version = "2.9"
bracken_dir = f"/tmp/Bracken-{bracken_version}"
bracken_install_dir = "/usr/local/bracken"

# Clean up any existing Bracken installation
!rm -rf {bracken_dir} {bracken_install_dir} /tmp/bracken.tar.gz

# Download and extract Bracken
!wget https://github.com/jenniferlu717/Bracken/archive/refs/tags/v{bracken_version}.tar.gz -O /tmp/bracken.tar.gz
!tar -xzf /tmp/bracken.tar.gz -C /tmp

# Compile kmer2read_distr
!cd {bracken_dir}/src && make

# Create installation directory and copy files
!mkdir -p {bracken_install_dir}
!cp {bracken_dir}/bracken {bracken_install_dir}/
!cp {bracken_dir}/bracken-build {bracken_install_dir}/
!cp {bracken_dir}/src/est_abundance.py {bracken_install_dir}/
!if [ -f {bracken_dir}/src/kmer2read_distr ]; then cp {bracken_dir}/src/kmer2read_distr {bracken_install_dir}/; fi
!chmod +x {bracken_install_dir}/bracken {bracken_install_dir}/bracken-build {bracken_install_dir}/est_abundance.py
!if [ -f {bracken_install_dir}/kmer2read_distr ]; then chmod +x {bracken_install_dir}/kmer2read_distr; fi

# Verify Bracken installation
required_bracken_files = ["bracken", "bracken-build", "est_abundance.py"]
for f in required_bracken_files:
    if not os.path.exists(os.path.join(bracken_install_dir, f)):
        raise FileNotFoundError(f"Bracken file {f} not found at {bracken_install_dir}/{f}")

# Install required Python dependencies for Bracken
!pip install numpy

# Confirm installs
!which vsearch
!vsearch --version
!which kraken2
!kraken2 --version
!which fastp
!fastp --version
!{bracken_install_dir}/bracken -v

# Setup Kraken2 DB
db_dir = "/kaggle/tmp/kraken2_db"
os.makedirs(db_dir, exist_ok=True)

# Download Standard 8GB Kraken2 DB
db_tar = f"{db_dir}/k2_standard_08gb_20240605.tar.gz"
if not os.path.exists(f"{db_dir}/hash.k2d"):
    !wget -c https://genome-idx.s3.amazonaws.com/kraken/k2_standard_08gb_20240605.tar.gz -O {db_tar}
    !tar -xvzf {db_tar} -C {db_dir}

# Set the DB path
db_path = db_dir

# Verify DB files
required_files = ["hash.k2d", "taxo.k2d", "opts.k2d", "database150mers.kmer_distrib"]
if not all(os.path.exists(os.path.join(db_path, f)) for f in required_files):
    raise FileNotFoundError(f"Required DB files {required_files} not found in {db_path}")

# Set environment variable for Kraken2
os.environ["KRAKEN2_DB_PATH"] = db_path

In [ ]:
# ================= Kaggle Setup for vsearch + Kraken2 =================
import os
import subprocess
import glob
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, as_completed
import logging
import gzip

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
fh = logging.FileHandler('/kaggle/working/process_log.txt')
fh.setLevel(logging.INFO)
logger.addHandler(fh)

def run_shell(cmd, check=True):
    """Run a shell command and return (exit_code, stdout, stderr)."""
    proc = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    if check and proc.returncode != 0:
        raise RuntimeError(f"Command failed: {cmd}\nstdout: {proc.stdout}\nstderr: {proc.stderr}")
    return proc.returncode, proc.stdout, proc.stderr

def get_read_length(fastq_file):
    """Estimate read length from FASTQ or FASTQ.gz file."""
    try:
        if fastq_file.endswith('.gz'):
            with gzip.open(fastq_file, 'rt') as f:
                f.readline()  # header
                seq = f.readline().strip()
        else:
            with open(fastq_file, 'r') as f:
                f.readline()
                seq = f.readline().strip()
        return len(seq)
    except:
        return 150  # fallback

def process_sample(sample_id, fastqs, db_path, output_csv, write_header=False):
    """Process a single sample with Kraken2 and write result directly to CSV."""
    report_path = f"/tmp/{sample_id}.report"
    kraken_out = f"/tmp/{sample_id}.kraken"
    try:
        # Input checks
        for fq in fastqs:
            if not os.path.exists(fq):
                raise FileNotFoundError(f"Input FASTQ file {fq} not found")
            if os.path.getsize(fq) == 0:
                raise ValueError(f"Input FASTQ file {fq} is empty")

        # Kraken2 command
        paired_flag = "--paired" if len(fastqs) == 2 else ""
        compressed_flag = "--gzip-compressed" if any(fq.endswith('.gz') for fq in fastqs) else ""
        input_files = " ".join(fastqs)

        cmd_kraken = (
            f"kraken2 --db {db_path} {paired_flag} {compressed_flag} {input_files} "
            f"--output {kraken_out} --report {report_path} "
            f"--threads 2 --use-names --report-zero-counts --memory-mapping"
        )
        run_shell(cmd_kraken)

        # Parse report
        if not os.path.exists(report_path) or os.path.getsize(report_path) == 0:
            raise RuntimeError(f"No valid report for {sample_id}")

        df = pd.read_csv(
            report_path, sep="\t", header=None,
            names=["percent", "reads_rooted", "reads_direct", "rank", "taxid", "name"],
            dtype={"name": str}
        )
        df2 = df[df["rank"] == "S"][["name", "reads_rooted"]].copy()
        df2["name"] = df2["name"].str.strip()
        df_wide = df2.set_index("name")["reads_rooted"].to_frame().T
        df_wide["SampleID"] = sample_id

        # Append to CSV
        df_wide.to_csv(output_csv, mode="a", index=False, header=write_header)

        return True

    except Exception as e:
        logger.error(f"Error processing {sample_id}: {e}")
        return False
    finally:
        for f in [report_path, kraken_out]:
            if os.path.exists(f):
                os.remove(f)

In [ ]:
# ——————————————————————————————
# Step 0: Setup
# ——————————————————————————————
working_dir = "/kaggle/working"
os.makedirs(working_dir, exist_ok=True)
output_csv = os.path.join(working_dir, "microbes_abundances.csv")

db_path = os.environ.get("KRAKEN2_DB_PATH", "/kaggle/tmp/kraken2_db")
if not os.path.exists(os.path.join(db_path, "hash.k2d")):
    raise FileNotFoundError(f"Kraken2 DB not found at {db_path}")

# ——————————————————————————————
# Step 1: Find FASTQ samples
# ——————————————————————————————
r1_files = glob.glob("/kaggle/input/**/*_R1*.fastq*", recursive=True)
samples = defaultdict(list)
for r1 in r1_files:
    sample_id = os.path.basename(r1).split("_R1")[0]
    r2 = r1.replace("_R1", "_R2")
    samples[sample_id] = [r1, r2] if os.path.exists(r2) else [r1]

other_fastq = set(glob.glob("/kaggle/input/**/*.fastq*", recursive=True)) - set(sum(samples.values(), []))
for fq in other_fastq:
    sample_id = os.path.basename(fq).split(".")[0]
    samples[sample_id] = [fq]

logger.info(f"Found {len(samples)} samples to classify.")

# ——————————————————————————————
# Step 2: Process sequentially (low memory)
# ——————————————————————————————
first = True
success_count = 0

for sample_id, fastqs in tqdm(samples.items(), desc="Classifying samples"):
    ok = process_sample(sample_id, fastqs, db_path, output_csv, write_header=first)
    if ok:
        success_count += 1
        first = False  # only write header once

logger.info(f"Finished classification. {success_count}/{len(samples)} samples processed successfully.")
logger.info(f"Results saved to: {output_csv}")